<a href="https://colab.research.google.com/github/Vishwakbalaji/Google_colab/blob/main/stage1%262_Capp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
#2.1 Get the file
downloaded = drive.CreateFile({'id':'1Y4aZx_NUolNj-HjOD6zKP4j6lAFUWuiK'}) # replace the id with id of file you want to access
downloaded.GetContentFile('WITHOUT_duplicate.csv')

In [4]:
pip install pattern

In [5]:
import numpy as np
import pandas as pd
import os
import string
string.punctuation
import re
from textblob import TextBlob # using text blob function
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
import gensim
from pattern.text.en import singularize

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
def entier_func(data,column_drop,col_name):
    
    df = pd.read_csv(data)
    df.drop('Unnamed: 0',axis=1,inplace=True)
    df.dropna(axis=0,inplace=True)
    df1 = df.drop(column_drop,axis=1)
    
    # pre-processing:
    
    # punctuation removal
    def remove_punctuation(text):
        punctuation_free="".join([i for i in text if i not in string.punctuation])
        return punctuation_free
    
    df1['Review_no_punc']=df1[col_name].apply(lambda x: remove_punctuation(x))
    
    # lowering text
    df1['Review_lower']= df1['Review_no_punc'].apply(lambda x: x.lower())
    
    # extra white space removal
    df1['Review_no_extra_space'] = df1['Review_lower'].apply(lambda text: re.sub(' +', ' ', text))
    
    # emoji removal
    def emoji(string):  # created a emoji removing funcction
        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', string)
    
    df1['Review_no_emoji'] = df1['Review_no_extra_space'].apply(emoji)
    
    
    # Tokenization:
    def tokenization(text):
        tokens = re.split('W+',text)
        return tokens
    
    a = df1['Review_no_emoji'].apply(lambda x: tokenization(x)) # calling the function
    df1['Review_token'] = a.apply(lambda x: " ".join(x)) # Join the words into the String
    
    # remove the words less than two words:
    def greater_than_2(text):
        result = []
        for token in gensim.utils.simple_preprocess(text):
            if len(token) > 3:
                result.append(token)
        return result
    
    b = df1['Review_token'].apply(lambda x: greater_than_2(x)) 
    df1['Review_above_3token'] = b.apply(lambda x: " ".join(x)) # Join the words into the String
    # Necessary Tagging:
    
    # parts of speech tagging
    def pos_tag(text):
        try:
            return TextBlob(text).tags
        except:
            return None
    
    df1['Review_pos_tag'] = df1['Review_above_3token'].apply((pos_tag)) # calling pos_tag function 
    
    # addjective tagging
    def get_adjectives(text):
        blob = TextBlob(text)
        return [ word for (word,tag) in blob.tags if tag == "JJ"]
    
    df1['Review_adjective'] = df1['Review_above_3token'].apply(get_adjectives)
    
    # Common Noun(NN and NNS) [which is called Noun and as well as common noun and NNP, NNPS is called proper Noun]
    def get_common_nouns(text):
        blob = TextBlob(text)
        return [ word for (word,tag) in blob.tags if tag == "NN" or tag == "NNS"]
    
    df1['Review_Common_Noun'] = df1['Review_above_3token'].apply(get_common_nouns)
    df1.reset_index(inplace = True, drop = True) # changed the index structure
    
    # converting it into singular
    def singularity(texts):
        return [singularize(text) for text in texts]
    # calling the function and saving it as like singular noun
    df1['Review_Singular_Cnoun'] = df1['Review_Common_Noun'].apply(lambda x: singularity(x)) # calling the function
    
    return df1

In [7]:
df1 = entier_func(data = "WITHOUT_duplicate.csv",column_drop = ['star','hotel_name'],col_name = 'review')
df1.head()

,review,Review_no_punc,Review_lower,Review_no_extra_space,Review_no_emoji,Review_token,Review_above_3token,Review_pos_tag,Review_adjective,Review_Common_Noun,Review_Singular_Cnoun
0,"Good hotel, great staffs, nice food and servic...",Good hotel great staffs nice food and service ...,good hotel great staffs nice food and service ...,good hotel great staffs nice food and service ...,good hotel great staffs nice food and service ...,good hotel great staffs nice food and service ...,good hotel great staffs nice food service near...,"[(good, JJ), (hotel, NN), (great, JJ), (staffs...","[good, great, nice]","[hotel, staffs, food, service, beach, shopping...","[hotel, staff, food, service, beach, shopping,..."
1,"Very good hotel, nice service we will plan nex...",Very good hotel nice service we will plan next...,very good hotel nice service we will plan next...,very good hotel nice service we will plan next...,very good hotel nice service we will plan next...,very good hotel nice service we will plan next...,very good hotel nice service will plan next va...,"[(very, RB), (good, JJ), (hotel, NN), (nice, J...","[good, nice, next, friendly]","[hotel, service, vacation, family, friends, st...","[hotel, service, vacation, family, friend, sta..."
2,Far away from the over crowded Calangute and B...,Far away from the over crowded Calangute and B...,far away from the over crowded calangute and b...,far away from the over crowded calangute and b...,far away from the over crowded calangute and b...,far away from the over crowded calangute and b...,away from over crowded calangute baga whisperi...,"[(away, RB), (from, IN), (over, IN), (crowded,...","[crowded, candolim, perfect, commercial, allot...","[calangute, baga, palms, location, access, gat...","[calangute, baga, palm, location, acces, gate,..."
3,Very close to Candolim beach. Nice food and bu...,Very close to Candolim beach Nice food and buf...,very close to candolim beach nice food and buf...,very close to candolim beach nice food and buf...,very close to candolim beach nice food and buf...,very close to candolim beach nice food and buf...,very close candolim beach nice food buffet swi...,"[(very, RB), (close, RB), (candolim, JJ), (bea...","[candolim, nice, good]","[beach, food, buffet, pool]","[beach, food, buffet, pool]"
4,"Very nice ambience, just 2 mins walk from reso...",Very nice ambience just 2 mins walk from resor...,very nice ambience just 2 mins walk from resor...,very nice ambience just 2 mins walk from resor...,very nice ambience just 2 mins walk from resor...,very nice ambience just 2 mins walk from resor...,very nice ambience just mins walk from resort ...,"[(very, RB), (nice, JJ), (ambience, NN), (just...","[nice, good, good]","[ambience, mins, resort, beach, rooms, food, p...","[ambience, min, resort, beach, room, food, pla..."


In [ ]:
# files for downloading
from google.colab import files
df1.to_csv('Hotel_Reviews_All_NLP_Process_file.csv') 
files.download('Hotel_Reviews_All_NLP_Process_file.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# files for downloding
from google.colab import files
df1['Review_Singular_Cnoun'].to_csv('Hotel_Reviews_singular_CN.csv') 
files.download('Hotel_Reviews_singular_CN.csv')

In [8]:
# Binary conversion:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
data = pd.DataFrame(mlb.fit_transform(df1['Review_Singular_Cnoun']),columns=mlb.classes_)

In [14]:
data

,aadesh,aage,aakanksha,aaloo,aamir,aana,aand,aapke,aapko,aaron,aasma,abatement,abbey,abbeyesque,abbott,abdirahman,abdul,abgood,abhibu,abhimanayu,abhishek,abience,abigail,abigel,ability,abit,ablation,able,abode,about,above,aboveaverage,abracadabra,absence,absent,absolute,absolutey,absorb,absoulute,absurd,...,yucknu,yumm,yummy,yumnroom,yvette,zaara,zaccuzi,zack,zahra,zahrannp,zara,zavus,zeal,zebop,zeebop,zeenla,zelo,zera,zero,zeroed,zeromaintenance,zestines,zillion,zinger,ziva,zizel,zofi,zoheb,zomato,zomatum,zombies,zone,zoop,zucca,zummer,zuri,zurich,zurus,zury,zwin
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58192,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
58193,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
58194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
58195,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
pd.set_option('display.max_columns', 15000)  # to display all the columns in the dataframe
pd.set_option('display.max_rows', 200)  # to display all the rows in the dataframe

In [19]:
data.iloc[0:1] # verifying it works or not. And it is working smoothly

,aadesh,aage,aakanksha,aaloo,aamir,aana,aand,aapke,aapko,aaron,aasma,abatement,abbey,abbeyesque,abbott,abdirahman,abdul,abgood,abhibu,abhimanayu,abhishek,abience,abigail,abigel,ability,abit,ablation,able,abode,about,above,aboveaverage,abracadabra,absence,absent,absolute,absolutey,absorb,absoulute,absurd,absurdity,abundance,abusive,abut,abuzz,acacium,academic,academy,acasium,acaustic,acce,accecibility,accent,accept,acceptance,acces,access,accessibility,accessible,accessing,accessory,accident,acco,accolade,accomadation,accomandation,accomdation,accomm,accommdation,accommodate,accommodating,accommodation,accomodate,accomodating,accomodatingnni,accomodation,accompaniment,accompany,accor,accord,accordance,accordionist,account,accountability,accounting,acct,accumulate,accumulation,accurate,accuratenntldr,accuse,acer,aces,acfan,acfridgeroom,ach,acha,ache,achha,achhe,achhi,achy,acility,acilitygaurav,acknowledge,acknowledgement,acknowledgment,aclas,acnnneed,acommodatio,acoomodate,acount,acoustic,acquaintance,acre,acron,acros,acrossthey,act,actin,action,activa,activate,active,activenes,actividade,activite,activitieskid,activity,actual,actualniranjan,actualy,actv,actvlight,adam,adamant,adaptability,addict,addition,addon,addres,adequate,adequateni,adherent,adhoc,adil,adip,aditus,aditya,adjacent,adjective,adjoin,adjoining,adjust,adjustable,adjusting,adjustment,adlina,admin,administration,adminop,admirable,admiration,admire,admission,admit,admittance,admuthe,adolescent,adult,advait,advance,advani,advantag,advantage,advantege,adventure,adventuredance,advertise,advertisement,advertising,advice,advisable,advise,advisor,advisory,advocate,adyant,aesthetic,aeverage,aferward,afety,affair,affect,affection,affectionate,affiliation,affirm,afford,affordability,affordable,afraid,aftab,afternoon,afterthought,afterward,aftet,aftr,agadi,agai,againand,againit,againnni,againnthank,againt,againwa,agar,agarwal,agatha,agauda,age,agency,agenda,agent,agethe,aggarwal,agoda,agonda,agood,agor,agouda,agoyashodan,agra,agree,agreed,agreement,aguada,aguavada,agud,aguda,ahem,ahluwalium,ahmedabad,aid,aint,aiport,airbreeze,aircon,aircondition,airconditioner,airconditioning,aire,airflow,airfort,airgun,airline,airplane,airpod,airpor,airport,airportand,airportcity,airportn,airportpickup,airportrail,airportroli,airporttrain,airpot,airtel,airway,airwheel,airy,aishwarya,aisle,aiswarya,aiysha,ajar,ajay,ajit,ajith,ajuna,aka,akaran,akash,akella,aker,akhil,akkdam,akshay,akso,ala,alacard,alacarte,alacartum,alacatre,alam,alarm,alarmu,alavla,alaykumar,albeit,albiet,album,alcarte,alchemist,alchohol,alcohol,aldeium,alert,alexa,alexander,alfredo,alga,alien,alight,alila,aline,alisha,alistair,alitymallica,alium,alizabeth,alka,all,alla,allaamir,allergic,allergy,alley,alleyway,allhello,alliance,allin,allina,allinone,allkaran,alll,allni,allnnu,allno,allocate,allocation,allot,allotment,allout,allover,allow,allowance,allows,allperson,allround,allshiv,allshwetum,allthe,allure,alluring,allwater,ally,allyl,alma,almirah,almond,alo,aloft,alona,alone,alonegood,alongood,alongside,alongwith,aloo,aloof,aloofnes,alot,alpa,alread,alredy,alright,alryt,alsoafter,alson,alsonvery,alsootherwise,alter,altercation,alternate,alternative,althogh,altogether,altura,aluminium,alurium,alush,alva,alvaro,amabiance,aman,amanety,amar,amarande,amarante,amardeep,amarinder,amaro,amarth,amateur,amaz,amaze,amazin,amazing,amazinggarima,amazingnreview,amazingu,amazon,ambassador,ambeince,ambiance,ambiancen,ambiancennthe,ambiebce,ambiemce,ambienc,ambience,ambiencesafe,ambiencesohini,ambiencshashank,ambient,ambiguity,ambition,ambulance,amekity,amending,amendment,amenety,amenitie,amenitiesi,amenitiesn,amenitiesnmake,amenitiespool,amenitiesvery,amenity,amentiny,amentity,amenty,amerante,america,americinn,amey,amibeyance,amicable,amidst,amimpressive,amin,amineti,aminite,aminity,aminul,amit,amitabh,amitava,amizadesna,ammenety,ammenity,ammentity,ammenty,amminity,ammunity,amneity,amnesty,amnety,amol,amongst,amont,amount,amout,ample,amplif

In [20]:
data.shape

(58197, 15426)

In [9]:
# downloading the binary file
from google.colab import files
data.to_csv('Hotel_Reviews_Binary_Conversion.csv') 
files.download('Hotel_Reviews_Binary_Conversion.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# creating a unique list for checking purpose
text = df1['Review_Singular_Cnoun']
unique_noun_list = []   # it is for unique words
for i in range(0,len(text)): # iterating the len of noun column
      for j in text[i]:        # iterating the text of each index in noun column
          if j not in unique_noun_list:      # if the iterated text not presented inside the list
              unique_noun_list.append(j)     # then we append it into the list
          else:                              # if not we leave it and continue
              continue

In [11]:
# executing the above loop
pd.DataFrame(columns=unique_noun_list)

,hotel,staff,food,service,beach,shopping,market,vacation,family,friend,hygienic,tasty,calangute,baga,palm,location,acces,gate,part,concern,room,block,pool,restaurant,activity,area,facing,event,night,evening,house,rainwater,govind,singh,understood,suite,balance,breakfast,taste,variety,...,foodnwe,talkative,legion,roomsat,plasticnnwhen,towelsnnid,brisk,bistrou,ballgame,redevelopment,qaccommodating,fringe,staffnwe,morennwa,nnwouldnu,staywhich,raquetball,admittance,dumbbell,cinsession,cleannnwe,hotelgreat,bageln,calgaryu,prairie,toiletu,locationwe,helpfulni,migration,molelike,conventioneer,curmudgeon,loadingu,optionsnwater,getout,three,featery,buildup,smokingu,connecticut


In [12]:
# taking the single word from the multilable encoder output and checking the presence in the above unique list or not. if it present in the above unique list then it is working properly.
kkl = []
for i in unique_noun_list:
    if i == 'abhimanayu':
        kkl.append(i)
    else:
        continue

In [13]:
kkl # it is working properly(because in above condition if that word present in unique list then append it into kkl)

['abhimanayu']